# Customer Retention in Telecommunication Company

**Amy Robertson 12564896**

**Carmen Chan 12614246**

Link to this report: https://github.com/carmen-chan/A2/blob/master/A2%20Report%20Machine%20Learning%20Spring.ipynb

Link to code: https://github.com/carmen-chan/A2/blob/master/Code%20For%20Assignment%202.ipynb

Link to video: https://screencast-o-matic.com/watch/cqQTi0uVVQ


# Introduction

The world of telecommunications is changing - mobile phones and the accessibility of mobile data has changed what companies can offer and what customers value. As the world becomes more interconnected customers have higher demands for internet service and lowered use of traditional phones. Due to this shift, new companies are emerging that offer low prices on data only plans which threaten the traditional telecommunication companies (telco). As competition grows retaining current customers is increasingly important - understanding what different customers are using in a telco service and what are important to offer can help telco companies develop strategies to improve retention. This could be through personalising customers experiences by offering specific deals just prior to a churn period or specialising a plan so it includes items that often-retained customers enjoy.

This report examines a telco dataset of 7043 customers with 21 attributes that include customer information and service subscriptions with the target variable 'churn' which is the monthly retention rates. Using machine learning algorithms - decision tree, random forest, Linear SVM and poly Kernel SVM - we will classify which customers will churn or be retained which demonstrates the type of customers that are satisfied with their service and those who are not so we may learn from them to increase retention.  





# Exploration

The telco dataset used in this report comes from the website Kaggle - it is uploaded by BlastChur two years ago and includes 21 attributes and 7043 rows. The data is not sourced however mentions IBM tests, therefore there is ambiguity around where the data has been sourced and the integrity of it. However for the purposes of understanding customer churn in telecommunication companies it offers value to understand how different attributes work in it. 

The original dataset includes customerID as the first attribute - for the purposes of this report it will be ignored as it does not add value to the results of classification. 

The first step in tackling this problem was to understand the dataset itself. This includes 

Below is a brief summary of each attribute and how they are distributed:

*   *'Gender'* - string(object) - 49.5% Female, 50.5% Male.
*   *'Senior Citizen'* - Binary - 16.2% are senior citizens, 83.8% are not. 
* *'Partner'* - String(object) -  48.3% have a partner, 51.7% do not
* *'Dependents'* - String(object) - only 29.96% have dependents
* *'Tenure'* - Float(int64) - fairly normally distributed, slightly right skewed. Mean of 33.4 months with a maximum of 72 and minimum of 0. The standard deviation is 24.6. 

![Graph of Tenure with right skew](https://i.imgur.com/QD57TDE.jpg)
* *'Phone Service'* - String(object) - 90% have a phone service, 10% do not
* *'MutlipleLines'* - String(object) - 48% do not have multiple lines, 42.2% do and 10% do not have a phone service.
* *'InternetService'* - String(object) - 43.9% Fibre Optic, 34.4% DSL and 21.7% have none
* *'Online Security'* - String(object) - 28.7% Do, 49.67% do not, remaining 21.7% have no internet
* *'OnlineBackup'* - String(object) -  34.5% do, 43.8% Do not, remaining 21.7% have no internet
* *'DeviceProtection'* - String(object) - 34.4% do, 43.9% do not, remaining 21.7% have no internet
* *'TechSupport'* - String(object) - 29% subscribe to technical support, 49.3% don't, remaining 21.7% have no internet
* *'StreamingTV'* - String(object) - 38.4% have tv streaming services, 39.9% don't, remaining 21.7% have no internet
* *'StreamingMovies'* - String(object) - 38.8% have movie streaming services, 39.5% don't, remaining 21.7% have no internet
* *'Contract'* - String(object) - 55% Month to Month, 20.9% One year, 24% Two year
* *'PaperlessBilling'* - String(object) - 59.2% use paperless billing, 30.8% do  not.
* *'Payment Method'* - String(object) - 21.9% AutoBank, 21.6% CreditCardAuto, 33.6% Electronic check, 22.9% Mailed Check
* *'MonthlyCharges'* - Float(int64) - Fairly normally distributed with a slight left skew. Mean of 64.8 with a maximum of 118.75 and a minimum of 18.3. The standard deviation is 30.1. 

![Graph of Monthly Charges with left skew](https://i.imgur.com/VprAPYf.jpg)
* *'Totalcharges'* - Float(int64) - Strong right skew. Mean of 2283.3, with a maximum of 8684.8 and minimum of 18.8. The standard deviation is 2266.8. 

![Graph of Total Charges with left skew](https://i.imgur.com/8o5nkd9.jpg)
* *'Churn'* - Skewed target data so accuracy statistic won't be useful - need F score instead. 73.46% is No (as in retained) and 26.54% is yes (as in lost)


The attributes were evaluted in KNIME for relationships and produced this correlation matrix. 

![Correlation Matrix](https://i.imgur.com/ynrVAQJ.jpg)

The interesting relationships are outlined below: 
* *'StreamingTV'* positively correlates strongly with Device *'Protection'* (0.76), *'InternetService'* (0.71) and *'OnlineSecurity'* (0.7) 
* StreamingMovie positively correlates strongly with *'DeviceProtection'* (0.77), *'InternetService'* (0.71) and *'OnlineSecurity'* (0.7)
* *'StreamingMovies'* and *'StreamingTV'* positively correlate strongly with each other (0.83)
* *'Totalcharges'* positively correlates strongly with *'Tenure'* (0.81) 
* *'Onlinesecurity'* positively correlates with *'TechSupport'* (0.79) 
* *'Onlinebackup'* negatively correlates with *'MonthlyCharges'* (-0.71)
* The attributes that correlated most with the target *'Churn'* are:
  * Negatively with Contract, TenureMonths, OnlineSecurity, TechSupport, DeviceProtection
  * Positively MonthlyCharges, Senior Citizen and Partner
  * It correlates least with Gender, PhoneService and InternetService
  
  
The dataset is reasonably clean with little to no missing values and no outliers. Therefore the data did not require high levels of cleaning. 

The data did require augmentation as majority of the attributes were strings. These attributes were encode into categories that are numerical so models can input this data. This was critical to begin experimenting with the dataset in any model. There were two options to do this - onehotencoder which splits the data into binary attributes for each potential category across columns and and label encoder which simply assigned a numeric category rather than a string. This report used the label encoder as the models we are using for classification are robust enough to work with this kind of sequential categorical attribute (Scikit Learn n.d. a). 
 

Additionally *'TenureMonths', 'MonthlyCharges' and 'TotalCharges'* required normalisation to ensure that they were easier to interpret by models as they currently have a much larger range - with *'TotalCharges'* reaching nearly 9000. A standard scalar was used from the sklearn pre-processing package was used to normalise these attributes for the models to compute these datasets better and it did increase the model accuracies (Scikit Learn n.d. b). 



  



In [0]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
telco = telco.apply(LabelEncoder().fit_transform)


scaler = preprocessing.StandardScaler()
scaled_telco = scaler.fit_transform(ColumnstoScale)
scaled_telco = pd.DataFrame(scaled_telco, columns=['TotalCharges_Normalised', 'MonthlyCharges_Normalised', 'TenureMonths_Normalised'])

The models we have decided to use are Decision Tree, Random Forest, SVM Linear & Poly Kernel. 

**Decision Tree**

We have used a decision tree as we deal with a discrete target variable and the data is fairly logical in nature. The decision tree is a quality model as it is robust to errors and can work around data that may still be messy in nature. The concern with using a decision tree is how prone it is to overfitting to the dataset (DeZyre 2016). 


**Random Forest** 

Random Forest classifiers are many decision trees and therefore has the same advantages, it also improves resistance to overfitting and often has increased accuracy as a result of completing multiple decision tree models (DeZyre 2016). 


**SVM**

Support Vector Machine classifiers can capture complicated data and transform it to compute complex relationships. It is useful in both regression and classification environments. Whilst it might be more than enough for the simple dataset used in this report it is still useful for comparison to the other two very simple algorithms mentioned above (DeZyre 2016).
This report uses both linear - where the data is split in the new space by a straight linear line - and Poly kernel - where the boundaries of clasification are created through clusters and are non linear. 


# Methodology

The models used are all imported from the sklearn package for python coding and are limited to the parameters available for adjustment within these. 

The first step after cleaning the data so it would work with the models which is mentioned in the exploration section of the report. 
Once this is complete the data is split into the training and test datasets. First by allocating which attributes will be used for prediction. The following attributes were used for prediction: 

* 'Gender', 'SeniorCitizen', 'Partner','Dependents', 'TenureMonths_Normalised',  'PhoneService',  'MultipleLines',  'InternetService',  'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'ContractTerm', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges_Normalised', 'TotalCharges_Normalised'

The target is then set to be 'MonthlyChurn'. Once these have been defined the training and test sets are split with an 80/20 split. 

After this the classifiers were imported from sklearn their parameters adjusted as below: 

* **Decision Tree** (Navlani 2018)
  * Experimented with different max_depth options to find that 6 is the optimal depth for this dataset classification. 

In [0]:
from sklearn.tree import DecisionTreeClassifier 
dtC = DecisionTreeClassifier(criterion="entropy", max_depth=6)

dtC = dtC.fit(X_train,y_train)

y_pred_dtC = dtC.predict(X_test)

print('Weighted F1 Score', f1_score(y_test, y_pred_dtC, average='weighted') )
print("Accuracy of decision tree classifier:",metrics.accuracy_score(y_test, y_pred_dtC))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_dtC))
print(classification_report(y_test,y_pred_dtC))
print(confusion_matrix(y_test,y_pred_dtC))


* **Random Forest** (Malik 2018a)
  * The n_estimators, max_depth and random_state values were found through trial and error to find these optimal values. Higher number of estimators may improve accuracy however it increases computational cost. 

In [0]:
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier(n_estimators=500, max_depth=7, random_state=0 )
randomforest.fit(X_train, y_train)
y_pred_randomforest = randomforest.predict(X_test)


print('Weighted F1 Score', f1_score(y_test, y_pred_randomforest, average='weighted') )
print(confusion_matrix(y_test,y_pred_randomforest))
print(classification_report(y_test,y_pred_randomforest))
print("Accuracy of random forest classifier:", accuracy_score(y_test, y_pred_randomforest))


* **Linear SVM** (Malik 2018b) 
  * Did not alter any parameters once set to linear. 

In [0]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred_SVM = svclassifier.predict(X_test)


print('Weighted F1 Score', f1_score(y_test, y_pred_SVM, average='weighted') )
print(confusion_matrix(y_test,y_pred_SVM))
print(classification_report(y_test,y_pred_SVM))
print("Accuracy of SVM classifier:", accuracy_score(y_test, y_pred_SVM))

* **Poly Kernel SVM** (Malik 2018b)
  * Set degree to 1 after trial and error found this to be the optimal degree for best results. 

In [0]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=1)
svclassifier.fit(X_train, y_train)

y_pred_SVK = svclassifier.predict(X_test)


print('Weighted F1 Score', f1_score(y_test, y_pred_SVK, average='weighted') )
print(confusion_matrix(y_test,y_pred_SVK))
print(classification_report(y_test,y_pred_SVK))
print("Accuracy of SVC classifier:", accuracy_score(y_test, y_pred_SVK))

Attributes with low correlation were also removed to test to see if that simplification improved the classifiers. It seemed to improve the decision tree by 0.013679 (rounding) and SVM linear by 0.003085 (rounding) classifiers, however, worsen the random forest by 0.007144 (rounding) and kernel SVM by 0.00018 (rounding) 

# Evaluation

Initially errors occurred when executing the models as the data was not clean enough or the cleaning code cells were not executed prior properly. However after encoding the object attributes effectively all models executed appropriately. 
Each model prints the weighted F1 score and a classification report. 


The importance of using the weighted F1 score is due to the target variable being imbalanced therefore it is easy for the models to achieve a high accuracy score without actually predicting the customers that churn as majority of the customers are retained (Shung 2018).

The F1 scores for each model are listed below: 
* Decision Tree - 0.7954053519036592
* *Random Forest - 0.8202317922734982*
* Linear SVM - 0.8002769583555663
* Poly Kernel SVM - 0.7994513080933549

Random Forest produces the best F1 score therefore would be the best model based on F1 score alone. However all four models do perform well within 3% of each other. 

Another evaluation of the models was an time efficiency analysis. This is computed using the python *'timeit'* method (Ardit 2019) to measure the time taken to output and produces the following outputs:
* Decision Tree - 0.2552287903000024
* Random Forest - 2.104208858419997
* Linear SVM - 1.4056256319600016
* Poly Kernel SVM - 0.9498196190299996

Decision Tree is the quickest model to execute, as it is the simplest of the models whilst the random forest is a 500 iterations of these decision trees therefore is computationally costly.




# Conclusion

The models can predict with around 80% accuracy if a customer will leave the telecommunication company. The recommended model for use is still the random forest classifier. It performed best with 2% more accuracy in the F1 Score than the others, whilst computationally most expensive it is not a complex model that has high resource demands. 

For future use more testing could be done to lower the computational cost without reducing the F1 score accuracy too significantly. However the Linear SVM also performed well in prediction and time efficiency. Additionally the dataset was potentially too simple to work on this complex problem. Gathering more data over time with additional information on each customer - for example their geographical location, income, time period (year-month joined and year-month left) and specific age - would assist to improve the model accuracy and remove any confounding variables that may be skewing the results. 

The models themselves work however more testing could be done with the additional models of K nearest neighbour and neural networks to deal with a potentially more complex data set if those additional attributes were collected.

# Ethics

 

With any introduction of technology into a business the ethical repercussions must be considered. Predicting whether a customer will leave a company may be beneficial for the company, however it may not have the same effect on the customer. 

Taking an initial ethical viewpoint of deontological those using this technology must always do the 'right' thing (BBC n.d. a). The way in which 'right' is defined could vary from a business perspective compared to human experience. A company can see it ‘right’ to predict the customer churn rate as it is essential to perform data analytics to help the company improve and grow. However, this 'right' may not be right for the customers. The hope would be that people would do the inherently right thing no matter what. However, this dichotomy between right for who may lead to disregarding consequences such as customer privacy or inaccurate predictions.

Another perspective is utilitarianism which adopts consequentialism to create the least negative impact for the most people possible (BBC n.d. b). Therefore, actions are always governed by the outcomes which cannot always be predicted. The drawback of utilitarianism in this environment is the difficulty when different people have different importance to those making the decisions - saving one person over many is an easy choice when you do not know the one person you have to sacrifice. In businesses the company can have more importance than the customers - therefore the actions taken should minimise negative effect in the company rather than customers or the planet. 


Misuses from this project could be that companies may try to reduce the customer churn rate in unethical ways. This could include offering the customers that are more likely to churn a different contract with higher charges or more strict lock in terms to make it more difficult for the customer to leave without communicating this clearly with the customer. Another ethically grey use of this knowledge could be to introduce specific exit fees which target individuals likely to churn and change contracts after a specific number of months when churn is more likely. These strategies are not illegal and would assist the business operations to increase retention, but they are not perceived as fair or ethical. It is often easy to use techniques such as this churn prediction to create more rigid and controlling contracts that force customers to stay. However, a more ethical strategy would be to improve the customer experience to help customers want to stay which could have the same business results.  

A middle ground between duty based deontological and utilitarian consequentialism should be always be found - where there is an inherent right and wrong that should be abided in most situations however the consequences of actions should still be considered prior to blindly following those rules.
 


# Reference List

Ardit, 2019, 'How to measure the execution time of a Python Script', *PythonHow.com*, 16 September, viewed 22 September, < https://pythonhow.com/measure-execution-time-python-code/ >. 

BBC n.d. a, *Duty-Based Ethics*, viewed 20 September 2019, < http://www.bbc.co.uk/ethics/introduction/duty_1.shtml >.

BBC n.d. b, *Consequentialism*, viewed 20 September 2019, < http://www.bbc.co.uk/ethics/introduction/consequentialism_1.shtml >.

DeZyre, 2016, *Top 10 Machine Learning Algorithms*, 29 Jan, viewed 13 September 2019, < https://www.dezyre.com/article/top-10-machine-learning-algorithms/202 >.

Malik U., 2018, 'Random Forest Algorithm with Python and Scikit-Learn', *StackAbuse*, 14 June, viewed 13 September 2019, < https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/ >.

Malik U., 2018, 'Implementing SVM and Kernel SVM with Python's Scikit-Learn', *StackAbuse*, 17 April, viewed 15 September 2019, < https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/ >.

Navlani A., 2018, 'Decision Tree Classification in Python', *DataCamp*, 29 December, viewed 13 September 2019, < https://www.datacamp.com/community/tutorials/decision-tree-classification-python >.

Scikit Learn, n.d. a, *sklearn.preprocessing.LabelEncoder*,  viewed 18 September 2019, < https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder > 

Scikit Learn, n.d. b, *sklearn.preprocessing.StandardScaler*,  viewed 18 September 2019, < https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler > 

Shung K. P., 2018, 'Accuracy, Precision, Recall or F1?', *Towards Data Science*, 15 March, viewed 18 September 2019, < https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9 >.

